In [ ]:
import pandas as pd
d = pd.read_excel("EUR/Polak-Kwartier-20190101.xlsx", names=["time", "kWh"], converters= {"Datum-tijd tot": pd.to_datetime}, skiprows=4, index_col=0)
d

In [ ]:
d["month"] = pd.DatetimeIndex(d.index).month 
d["year"] = pd.DatetimeIndex(d.index).year
d = d[d["year"]==2019]
d.tail(4)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig = plt.figure(figsize=(32, 12))
# fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(8,8))
# axs= axs.ravel()
# month_grps = d.groupby("month")

# for ax, (i, sub) in zip(axs, month_grps):
#     sub["kWh"].plot(kind="line", x="time", legend=True, title=i, ax=ax)
# monthly_means = d.resample("M").mean()
# print(monthly_means)
monthly_means = d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()
monthly_means = monthly_means.groupby(pd.Grouper(level='time', freq='M')).mean()
daily_mean_for_year = monthly_means.mean()
print(daily_mean_for_year)
# print(monthly_means)

for i in range(1, (12+1)):
    ax = fig.add_subplot(4, 3, i)
    # print(f"len is {len(d)}")
    plot_d = d[d["month"]==i]
    # monthly_mean = monthly_means[monthly_means["month"]==i]
    monthly_mean = monthly_means[monthly_means.index.month==i]
    mean = monthly_mean.iloc[0]
    # print(mean)
    # print(i, monthly_mean)
    # print(i, monthly_mean.head(1))
    # print(f"len is {len(plot_d)}")
    # print(plot_d.tail(2))
    # day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D')).sum()
    day_grps = plot_d.groupby(pd.Grouper(level='time', freq='D'))["kWh"].sum()

    # print(i)
    # print(day_grps)
    day_grps.plot(kind="line", y="kWh", ax=ax)
    # monthly_mean.plot(kind="line", ax=ax)
    ax.axhline(y=mean, color="g", alpha=0.5, linestyle="-")
    ax.axhline(y=daily_mean_for_year, alpha=1, color="y", linestyle="dotted")
    ax.legend(["daily kWh", "local daily avg", "global daily avg"])
    ax.set_xlabel("Day", labelpad=-3)
    ax.set_ylabel("Electricity Usage (kWh)")
    ax.grid()
    # ax.lin
plt.tight_layout()

In [ ]:
# Plotting GBS data 
# https://stackoverflow.com/questions/55776092/plotting-a-graph-by-month-wise-in-pandas
# https://stackoverflow.com/questions/61443949/python-pandas-group-by-then-plot-by-category

gbs_val_descriptions = {
    "FLOW":"airflow-sensor",
    "RS-CV":"regelsign-verwarming",
    "RS-GKW":"regelsign-koeling",
    "RS-VAV":"regelsign-vav", # Variable air volumewith 
    "PIR":"pir", # Infrared sensor
    "RM":"ruimte-modus",
    "RMT":"co2-ruimte",
    "SP":"setpoint-co2",
}

d = pd.read_excel("EUR/2022w18_GBS_trend_Y03-08.xlsx", parse_dates=[["Date", "Time"]], index_col=0)
print(d.columns)
# d.drop(["Y03VAV0339TE-01BSP.PointValue", 'Y03VAV0339TE-01CPA.PointValue'], axis=1, inplace=True)
d.drop(["Y03VAV0339TE-01BSP.PointValue", 'Y03VAV0339TE-01CPA.PointValue', 'Y03VAV0339TE-01RMT.PointValue', 'Y03VAV0339TE-01SP.PointValue'], axis=1, inplace=True)
# What is BSP and CPA?
print(d.columns)
d = d.rename(columns=lambda s: [value for key, value in gbs_val_descriptions.items() if key in s][0])
d.head(2)